In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import nltk.classify.util

from astropy.table import Table, Column
from datetime import timedelta

In [2]:
data = pd.read_csv('Amazon.csv')
print(data.shape)
data.head(5)

(455000, 13)


,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,helpScore,helpful
0,138806,138807,B000E63LME,A1CQGW1AOD0LF2,"Alena K. ""Alena""",1,2,2,1294185600,Not as pictured.,I was looking forward to try cranberry apple f...,0.5,False
1,469680,469681,B004ZIH4KM,A37S7U1OX2MCWI,Becky Cole,0,0,5,1349740800,seeds,"TY for everything. The seeds arrived quickly,...",NaN,False
2,238202,238203,B003ZXE9QA,A2OM6G73E64EQ9,jeff,0,0,5,1329264000,I'm addicted!,I've finally found the best cereal in the worl...,NaN,False
3,485307,485308,B001RVFERK,A25W349EE97NBK,Tangent4,1,1,4,1248307200,I wanted to love these...,I originally bought these chips because I'd he...,1.0,False
4,375283,375284,B000OQZNTS,A3CPPW0HUC07YS,Amy Nicolai,0,0,5,1333238400,Excellent chamomile tea,"Really excellent tea, flowers are visible in t...",NaN,False


In [3]:
# features from Amazon.csv to add to feature set

#features related to text body of review
data['reviewLen'] = data['Text'].str.len()
data['hasEP'] = data['Text'].str.contains('!')
data['hasQM'] = data['Text'].str.contains('\?')
data['hasAST'] = data['Text'].str.contains('\*')
data['hasDS'] = data['Text'].str.contains('$')
data['hasSC'] = data['Text'].str.contains(';')
data['hasAT'] = data['Text'].str.contains('@')
data['bodyNum'] = data['Text'].str.contains('\d')

data['EPcount'] = data['Text'].str.count('!')
data['QMcount'] = data['Text'].str.count('\?')
data['ASTcount'] = data['Text'].str.count('\*')
data['DScount'] = data['Text'].str.count('$')
data['SCcount'] = data['Text'].str.count(';')
data['ATcount'] = data['Text'].str.count('@')
data['NUMcount'] = data['Text'].str.count('\d')

#features related to profile name 
data['nameLen'] = data['ProfileName'].str.len()
data['nameNum'] = data['ProfileName'].str.contains('\d')
data['nameNumCount'] = data['ProfileName'].str.count('\d')
data['nameQUOTE'] = data['ProfileName'].str.contains('"')
data['nameSPACE'] = data['ProfileName'].str.contains(' ')

#time? sooner the better? 
data['first'] = data['Time'].rank(ascending=1)

#not sure if this is the best way to go about doing this - 
#finding repeating names 
data['repeatUser'] = data.duplicated('ProfileName')
data['repeatSum'] = data.duplicated('Summary')
data['repeatText'] = data.duplicated('Text')

data.tail(10)


,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,...,NUMcount,nameLen,nameNum,nameNumCount,nameQUOTE,nameSPACE,first,repeatUser,repeatSum,repeatText
454990,320405,320406,B000E1HVR0,A3CO3BDS4PXKL5,E. Morris,0,0,5,1249171200,OMG... need I say more,...,0,9,False,0,False,True,76987.5,True,False,False
454991,104610,104611,B000KSTY86,A3DTJW2ZSDQZ86,M. B. Goetz,11,11,5,1270080000,Great coffee - Great price,...,0,11,False,0,False,True,111697.0,True,False,False
454992,282272,282273,B000FICDO8,A3B5OLV01PVP85,"D. Patel ""SpeedReader""",1,1,5,1258675200,Great taste with little effort.,...,0,22,False,0,True,True,91108.5,False,False,False
454993,419683,419684,B0029ZAOW8,A2HAVA4SPL9HSM,"Larry A. Chrispyn ""PUBLISHED POET""",0,2,1,1329696000,PRICE TOO HIGH NO EFFECT,...,0,34,False,0,True,True,323594.0,True,True,True
454994,115270,115271,B003E50WMY,A37FP7HWWNTW57,Gary,0,0,5,1314921600,My Poinion.,...,0,4,False,0,False,False,243155.5,True,False,False
454995,457613,457614,B008Z5L2MW,A2W7HYIQWAJ091,"Bonnie Pierce ""TexasBonnieBelle""",0,0,5,1349049600,Outstanding - what else?,...,0,32,False,0,True,True,439001.0,True,False,False
454996,283509,283510,B0045Z4JAI,AZ3GK5ZX3SVKT,gsue,2,2,2,1306195200,"Too weak, not enough umph!",...,0,4,False,0,False,False,210328.0,True,True,True
454997,292569,292570,B001BM68S4,A359CZWDE8MPMF,LC,0,0,5,1314576000,Good Stuff,...,10,2,False,0,False,False,241547.5,True,True,False
454998,156159,156160,B002YLG5MA,A1V8J8FKBZW68K,"Ansella Adams ""Valley Lady""",4,4,5,1275350400,Way to Start the Day!,...,0,27,False,0,True,True,121137.5,False,False,False
454999,119066,119067,B004IREFUM,A1MYYCC9BSYV5Y,Colorado Mom,0,0,5,1338422400,Delicious,...,0,12,False,0,False,True,374840.5,True,True,True


In [4]:
#turns colum into vector - here columns 7 and 13
XScore = data.iloc[:, 7].values.reshape(data.shape[0], 1)
XreviewLen = data.iloc[:, 13].values.reshape(data.shape[0], 1)
XhasEP = data.iloc[:, 14].values.reshape(data.shape[0], 1)
XhasQM = data.iloc[:, 15].values.reshape(data.shape[0], 1)
XhasAST = data.iloc[:, 16].values.reshape(data.shape[0], 1)
XhasDS = data.iloc[:, 17].values.reshape(data.shape[0], 1)
XhasSC = data.iloc[:, 18].values.reshape(data.shape[0], 1)
XhasAT = data.iloc[:, 19].values.reshape(data.shape[0], 1)
XBodyNum = data.iloc[:, 20].values.reshape(data.shape[0], 1)
XEPcount = data.iloc[:, 21].values.reshape(data.shape[0], 1)
XQMcount = data.iloc[:, 22].values.reshape(data.shape[0], 1)
XASTcount = data.iloc[:, 23].values.reshape(data.shape[0], 1)
XDScount = data.iloc[:, 24].values.reshape(data.shape[0], 1)
XSCcount = data.iloc[:, 25].values.reshape(data.shape[0], 1)
XATcount = data.iloc[:, 26].values.reshape(data.shape[0], 1)
XNUMcount = data.iloc[:, 27].values.reshape(data.shape[0], 1)

XnameLen = data.iloc[:, 28].values.reshape(data.shape[0], 1)
XnameNum = data.iloc[:, 29].values.reshape(data.shape[0], 1)
XnameNumCount = data.iloc[:, 30].values.reshape(data.shape[0], 1)
XnameQUOTE = data.iloc[:, 31].values.reshape(data.shape[0], 1)
Xnamespace = data.iloc[:, 32].values.reshape(data.shape[0], 1)
Xfirst = data.iloc[:, 33].values.reshape(data.shape[0], 1)

XrepeatUser = data.iloc[:, 31].values.reshape(data.shape[0], 1)
XrepeatSum = data.iloc[:, 32].values.reshape(data.shape[0], 1)
XrepeatText = data.iloc[:, 33].values.reshape(data.shape[0], 1)

    

Xtoadd = np.concatenate((XScore, XreviewLen, XhasEP, XhasQM, XhasAST, XhasDS, XhasSC, XhasAT, XBodyNum, XEPcount, XQMcount, XASTcount, XDScount, XSCcount, XATcount, XNUMcount, 
XnameLen, XnameNum, XnameNumCount, XnameQUOTE, Xnamespace, Xfirst,XrepeatUser, XrepeatSum, XrepeatText ), axis=1)

In [5]:
# report on training and test sets
def print_results(model):
    #print('Error rate on training set: ')
    erTEST = ((y_train != y_pred).sum() / X_train.shape[0])
    #print('Accuracy rate on training set: ')
    AccTEST = (1 - (y_train != y_pred).sum() / X_train.shape[0])
    #print('True positive rate on training tet:')
    TruPosTEST = ((y_train==True) & (y_pred==True)).sum() / y_train.sum()
    #TruNegTEST = (((y_train==False) & (y_pred_train==False)).sum() / (y_train.shape[0] - y_train.sum()))
    #print('**************')
    #('Error rate on test set: ')
    erTRAIN = ((y_test != y_pred_test).sum() / X_test.shape[0])
    #print('Accuracy rate on test set: ')
    AccTRAIN = (1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    #print('True positive rate on test set')
    TruPosTRAIN = (((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    #print('True negative rate on test set')
    TruNegTRAIN = (((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))
    data_rows = [('Error Rate', erTEST, erTRAIN),
                 ('Accuracy Rate', AccTEST, AccTRAIN),
                 ('True Positives', TruPosTEST, TruPosTRAIN),
                 ('True Negatives', '--', TruNegTRAIN)]
    t = Table(rows=data_rows, names=(model, 'Training Set', 'Test Set'), meta={'name': model + ': Training and Test Set Results'})
    print(t)
    #t.show_in_browser(jsviewer=True) 
    

In [7]:
# vectorize Bag of Words from review text; as sparse matrix
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(min_df=1, ngram_range=(3,3))
X = tv.fit_transform(data.Text)

#if you restrict it to 2 to the power of 17 features is a few extra, way to restrict number of features created 

In [8]:
# convert additional features to sparse matrix and concatenate onto the bag of words sparse matrix
from scipy.sparse import csr_matrix, hstack
XtoaddSparse = csr_matrix(Xtoadd)
Xfinal = hstack([X, XtoaddSparse])
X = csr_matrix(Xfinal)

TypeError: Cannot cast array data from dtype('O') to dtype('bool') according to the rule 'safe'

In [9]:
# size of feature set
print(X.shape)

(455000, 10176762)


In [10]:
# define y
y = data.iloc[:, 12].values
y.shape

(455000,)

In [11]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [12]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [13]:
# MODEL: SVM, linear
#will also go through each row as many times as you want it to - default is 5, making changes to weights, trying to
#move them closer and closer to lessen cost i think

from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

#getting only 25% of positives correctly classified 

     SVM          Training Set      Test Set   
-------------- ----------------- --------------
    Error Rate 0.000894819466248 0.153333333333
 Accuracy Rate    0.999105180534 0.846666666667
True Positives    0.995092130188 0.449685220346
True Negatives                -- 0.878072304396


In [14]:
# MODEL: logistic regression
from sklearn import linear_model
#alpha, heavily penalizes the weights 
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set      Test Set   
------------------- ----------------- --------------
         Error Rate 0.000766091051805 0.144864468864
      Accuracy Rate    0.999233908948 0.855135531136
     True Positives    0.993628379542 0.432697112022
     True Negatives                -- 0.888555097911


In [15]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set   
-------------- ---------------- ---------------
    Error Rate 0.00194662480377 0.0805054945055
 Accuracy Rate   0.998053375196  0.919494505495
True Positives   0.999354227656  0.328969721195
True Negatives               --   0.96621156902


In [16]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set      Test Set   
-------------- ----------------- --------------
    Error Rate 0.000847723704867 0.152036630037
 Accuracy Rate    0.999152276295 0.847963369963
True Positives    0.994274151886 0.446487458779
True Negatives                -- 0.879724569739
